In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from ta.volatility import average_true_range as atr

SPX = yf.download('^GSPC', start='1990-01-01', end='2018-01-01')
RUA = yf.download('^RUA', start='1990-01-01', end='2018-01-01')
# price is already adjusted
SPX = SPX.drop(['Adj Close'], axis=1)
RUA = RUA.drop(['Adj Close'], axis=1)
# 10/200 MA
SPX['MA200'] = SPX['Close'].rolling(window=200).mean()
RUA['MA200'] = RUA['Close'].rolling(window=200).mean()
SPX['MA10'] = SPX['Close'].rolling(window=10).mean()
RUA['MA10'] = RUA['Close'].rolling(window=10).mean()
# ATR 10 for volatility
SPX['ATR10'] = atr(high=SPX['High'], low=SPX['Low'], close=SPX['Close'], window=10)
RUA['ATR10'] = atr(high=RUA['High'], low=RUA['Low'], close=RUA['Close'], window=10)
# Daily returns
SPX['Return'] = SPX['Close'].pct_change()
RUA['Return'] = RUA['Close'].pct_change()
# StDev of daily returns
SPX['StDev'] = SPX['Return'].std()
RUA['StDev'] = RUA['Return'].std()

# Side column if MA10 > MA200 1 (long), else -1 (short)
SPX['Side'] = np.where(SPX['MA10'] > SPX['MA200'], 1, -1) 
RUA['Side'] = np.where(RUA['MA10'] > RUA['MA200'], 1, -1)

# drop values for first 200 days since no MA200
SPX.dropna(inplace=True)
RUA.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
SPX

,Open,High,Low,Close,Volume,MA200,MA10,ATR10,Return,StDev,Side
Date,,,,,,,,,,,
1990-10-15,300.029999,304.790009,296.410004,303.230011,164980000,338.926850,306.849002,6.506506,0.010666,0.011078,-1
1990-10-16,303.230011,304.339996,298.119995,298.920013,149570000,338.623000,305.220004,6.477855,-0.014214,0.011078,-1
1990-10-17,298.920013,301.500000,297.790009,298.760010,161260000,338.323000,303.956006,6.201069,-0.000535,0.011078,-1
1990-10-18,298.750000,305.739990,298.750000,305.739990,204110000,338.073350,303.261005,6.279961,0.023363,0.011078,-1
1990-10-19,305.739990,312.480011,305.739990,312.480011,221480000,337.874750,303.359006,6.325967,0.022045,0.011078,-1
...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22,2684.219971,2685.350098,2678.129883,2683.340088,2399830000,2476.090654,2673.356030,15.262749,-0.000458,0.011078,1
2017-12-26,2679.090088,2682.739990,2677.959961,2680.500000,1968780000,2477.625804,2675.407031,14.274487,-0.001058,0.011078,1
2017-12-27,2682.100098,2685.639893,2678.909912,2682.620117,2202080000,2479.211655,2677.258032,13.520036,0.000791,0.011078,1


In [6]:
RUA

,Open,High,Low,Close,Volume,MA200,MA10,ATR10,Return,StDev,Side
Date,,,,,,,,,,,
1990-10-15,163.399994,165.610001,161.860001,165.009995,0,187.592550,167.328000,3.183242,0.009853,0.011119,-1
1990-10-16,165.009995,165.479996,162.410004,162.750000,0,187.404750,166.389999,3.171917,-0.013696,0.011119,-1
1990-10-17,162.750000,163.979996,162.199997,162.750000,0,187.217950,165.657999,3.032726,0.000000,0.011119,-1
1990-10-18,162.740005,166.399994,162.740005,166.399994,0,187.055700,165.241998,3.095452,0.022427,0.011119,-1
1990-10-19,166.399994,169.690002,166.399994,169.679993,0,186.919800,165.225998,3.114907,0.019712,0.011119,-1
...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22,1589.069946,1589.699951,1585.420044,1588.250000,0,1466.464298,1581.344006,9.168027,-0.000491,0.011119,1
2017-12-26,1587.520020,1588.300049,1585.630005,1587.000000,0,1467.373598,1582.690002,8.518229,-0.000787,0.011119,1
2017-12-27,1587.189941,1590.280029,1586.300049,1588.160034,0,1468.313948,1584.014001,8.064404,0.000731,0.011119,1
